In [1]:
import pandas as pd
import arrow
import datetime
import xlrd
from toolz.functoolz import pipe

In [2]:
file = "Group Data Updated 20180802.xlsx"

In [4]:
dfRaw = pd.read_excel(file, sheet_name="Sheet1")

book = xlrd.open_workbook(file)
datemode = book.datemode

Pipe lets us have a nice workflow where we just list the transformations we want, and our value will be "piped" sequentially through each one.



In [7]:
fns = [lambda x: xlrd.xldate_as_tuple(x, datemode),
     lambda x: x[3:5],
      lambda x: map(str, x),
      lambda x: "-".join(x),
       lambda x: arrow.get(x, "H-m"),
       lambda x: x.format('HH:mm:ss')
      ]
fnRanges = [fns[:i+1] for i in range(len(fns))]

Let's see a blow-by-blow of how one of our values gets transformed by that.

The first function takes us from an Excel datetime to a datetime tuple.

The next extracts just the time variables.

We then map that all to a string (which shows up as nothing because map is evaluated lazily).

Then we put a dash between all those elements so it'll be easier to parse as a time.

Then arrow consumes the value, with the format we specified.

And finally gives us a neatly-formatted time, ready to be consumed by a database!



In [6]:
def mapPipe(ser, fns):
    return ser.map(lambda a: pipe(a, *fns),
                   na_action="ignore" )

In [8]:
pd.DataFrame([mapPipe(dfRaw["Time of Appointment: Start"], 
       fns) for fns in fnRanges]).transpose().iloc[:5]

,Time of Appointment: Start,Time of Appointment: Start,Time of Appointment: Start,Time of Appointment: Start,Time of Appointment: Start,Time of Appointment: Start
0,"(0, 0, 0, 14, 30, 0)","(14, 30)",<map object at 0x7fcc37fd6860>,14-30,0001-01-01T14:30:00+00:00,14:30:00
1,"(0, 0, 0, 14, 0, 0)","(14, 0)",<map object at 0x7fcc37cf3dd8>,14-0,0001-01-01T14:00:00+00:00,14:00:00
2,"(0, 0, 0, 14, 0, 0)","(14, 0)",<map object at 0x7fcc37cf3c50>,14-0,0001-01-01T14:00:00+00:00,14:00:00
3,NaN,NaN,NaN,NaN,NaN,NaN
4,"(0, 0, 0, 11, 30, 0)","(11, 30)",<map object at 0x7fcc37cf31d0>,11-30,0001-01-01T11:30:00+00:00,11:30:00


In [27]:
dateFns = [lambda x: xlrd.xldate_as_tuple(x, datemode),
      lambda x: arrow.get(*x),
      lambda x: x.format('YYYY-MM-DD')
      ]

In [28]:
mapPipe(dfRaw['Date Payment Settled'],#.dropna(), 
       dateFns)

0             NaN
1             NaN
2             NaN
3             NaN
4      2017-06-04
5             NaN
6             NaN
7             NaN
8             NaN
9      2017-06-28
10            NaN
11            NaN
12            NaN
13            NaN
14            NaN
15            NaN
16            NaN
17     2017-03-10
18            NaN
19            NaN
20            NaN
21            NaN
22     2017-05-22
23            NaN
24            NaN
25            NaN
26            NaN
27            NaN
28            NaN
29     2016-12-23
          ...    
85            NaN
86            NaN
87            NaN
88            NaN
89     2017-08-30
90     2017-05-09
91            NaN
92            NaN
93            NaN
94            NaN
95     2017-06-06
96            NaN
97            NaN
98            NaN
99            NaN
100           NaN
101           NaN
102           NaN
103           NaN
104           NaN
105           NaN
106           NaN
107           NaN
108           NaN
109       

In [ ]:
pd.DataFrame([mapPipe(dfRaw["Date of Appointment"], 
       fns) for fns in fnRanges]).transpose().iloc[:5]

In [29]:
dateRange = [(2018, 8, 12, 21, 50, x) for x in range(16, 21)]

In [30]:
xlDates = [xlrd.xldate.xldate_from_datetime_tuple(x,
                                      0)
 for x in dateRange]

xlSer = pd.Series(xlDates)

In [31]:
excelTimes = [x-43324 for x in xlDates]

In [32]:
df = pd.DataFrame([(43324, x) for x in excelTimes],
            columns=["Date", "Time"])

In [ ]:
def printTable(ser):
    print(pd.DataFrame(ser).to_html())
printTable(df["Time"])